## Импорты

In [159]:
import random
from collections import deque, Counter
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

## Константы

In [160]:
# Типы карт
CARD_TYPES = [
    'crab', 'ship', 'fish', 'shark', 'swimmer',
    'shell', 'octopus', 'penguin', 'sailor'
]

# Стартовое количество карт
STARTING_COUNTS = {
    'crab': 9,
    'ship': 8,
    'fish': 7,
    'shark': 5,
    'swimmer': 5,
    'shell': 6,
    'octopus': 5,
    'penguin': 3,
    'sailor': 2,
}

# Парные и непарные карты
PAIRABLE = {'crab', 'ship', 'fish', 'shark', 'swimmer'}
UNPAIRED = {'shell', 'octopus', 'penguin', 'sailor'}

# Таблица очков
SHELL_SCORES = {1: 0, 2: 2, 3: 4, 4: 6, 5: 8, 6: 10}
OCTOPUS_SCORES = {1: 0, 2: 3, 3: 6, 4: 9, 5: 12}
PENGUIN_SCORES = {1: 1, 2: 3, 3: 5}
SAILOR_SCORES = {1: 0, 2: 5}

## Генерация базовой стратегии

In [161]:
def generate_strategy():
    top2 = ['penguin', 'ship']
    random.shuffle(top2)
    rest = [c for c in CARD_TYPES if c not in top2]
    random.shuffle(rest)
    return top2 + rest

## Генерация альтернативных стратегий

In [162]:
def reverse_strategy(strat):
    return strat[::-1]

def swap_paired_strategy(strat):
    new = strat.copy()

    paired_top2 = [c for c in new[:2] if c in PAIRABLE]
    paired = paired_top2[0]
    idx_p = new.index(paired)

    indices = [i for i, c in enumerate(new[2:], start=2) if c in PAIRABLE]
    i = random.choice(indices)

    new[idx_p], new[i] = new[i], new[idx_p]
    return new


def swap_unpaired_strategy(strat):
    new = strat.copy()

    unpaired_top2 = [c for c in new[:2] if c in UNPAIRED]
    up = unpaired_top2[0]
    idx_u = new.index(up)

    indices = [i for i,c in enumerate(new[2:], start=2) if c in UNPAIRED]
    i = random.choice(indices)

    new[idx_u], new[i] = new[i], new[idx_u]
    return new

## Подсчет очков в руке

In [163]:
def score_sets(hand):
    cnt = Counter(hand)
    score = 0
    score += SHELL_SCORES.get(cnt.get('shell', 0), 0)
    score += OCTOPUS_SCORES.get(cnt.get('octopus', 0), 0)
    score += PENGUIN_SCORES.get(cnt.get('penguin', 0), 0)
    score += SAILOR_SCORES.get(cnt.get('sailor', 0), 0)
    return score

## Симуляция раунда

### Вспомогательные функции для взятия карт

In [164]:
def get_from_discards(strat, discards):
    card = None
    for pref in strat:
        for p in (0, 1):
            if discards[p] and discards[p][-1] == pref:
                card = discards[p].pop()
                return card
    return card

def get_from_deck(strat, deck, discards, pick_one=True):
    # Если надо взять две карты и в колоде они есть
    if (not pick_one) and len(deck) >= 2:
        c1 = deck.popleft()
        c2 = deck.popleft()
        i1 = strat.index(c1)
        i2 = strat.index(c2)

        # Выбираем карту с большим приоритетом
        if i1 < i2:
            keep, drop = c1, c2
        else:
            keep, drop = c2, c1
        card = keep

        # Вторую карту в случайный сброс
        pile = random.choice((0,1))
        discards[pile].append(drop)
        return card
    
    # Если одна карта, то берем ее
    if deck:
        return deck.popleft()
    return None


def get_card(strat, deck, discards):
    # Ищем три приоритетные карты в сбросе
    card = get_from_discards(strat[:3], discards)
    
    # Если не нашли карту в сбросе, берем из колоды
    if card is None:
        card = get_from_deck(strat, deck, discards, False)
    return card

### Игровой цикл раунда

In [165]:
def play_round(strat1, strat2, stop_at_7=True, first_player=0):
    # Инициализируем и мешаем колоду
    deck = []
    for card, cnt in STARTING_COUNTS.items():
        deck += [card] * cnt
    random.shuffle(deck)
    deck = deque(deck)
    # Формируем сбрсы
    discards = [deque(), deque()]
    # Инициализируем руки и сыграные пары
    hands = [[], []]
    played_pairs = [0, 0]
    player = first_player
    turns = 0

    while True:
        turns += 1
        # Выбираем стратегию игрока
        strat = strat1 if player == 0 else strat2
        hand = hands[player]
        opp = 1 - player
        opp_hand = hands[opp]

        # Берем карту
        card = get_card(strat, deck, discards)
        if card is None:
            break
        hand.append(card)

        # Играем все возможные пары
        extra_turn = False
        made_pair = True
        while made_pair:
            made_pair = False
            cnt = Counter(hand)

            # Пары рыб, кораблей, крабов
            for p_card in ('fish', 'ship', 'crab'):
                if cnt[p_card] >= 2:
                    # Берем две одинакове карты
                    hand.remove(p_card)
                    hand.remove(p_card)
                    played_pairs[player] += 1
                    if p_card == 'fish':
                        if deck:
                            hand.append(deck.popleft())
                    elif p_card == 'ship':
                        extra_turn = True
                    elif p_card == 'crab':
                        card = get_from_discards(strat, discards)
                        if card:
                            hand.append(card)
                    made_pair = True
                    break
            if made_pair:
                continue

            # Акула и пловец
            if cnt['shark'] >= 1 and cnt['swimmer'] >= 1:
                hand.remove('shark')
                hand.remove('swimmer')
                played_pairs[player] += 1
                # Воруем случайную карты
                if opp_hand:
                    stolen = random.choice(opp_hand)
                    opp_hand.remove(stolen)
                    hand.append(stolen)
                made_pair = True

        # Проверяем условия окончания раунда
        score = played_pairs[player] + score_sets(hand)
        if stop_at_7 and score >= 7:
            break
        if not deck:
            break

        # Определяем, кто ходит
        if extra_turn:
            extra_turn = False
        else:
            player = opp

    # Итоговый счет
    scores = [played_pairs[i] + score_sets(hands[i]) for i in (0,1)]
    return scores, turns

## Симуляция игр

In [166]:
# Симуляция одной игры
def simulate_game(strat1, strat2, stop_at_7):
    # Раунд 1: Начинает первый игрок
    (s11, s12), t1 = play_round(strat1, strat2, stop_at_7, first_player=0)
    # Раунд 2: начинает второй игрок
    (s21, s22), t2 = play_round(strat1, strat2, stop_at_7, first_player=1)
    return s11 + s21, s12 + s22, t1 + t2

In [167]:
def simulate_n_games(strat1, strat2, stop_at_7, n_games):
    scores_1 = []
    scores_2 = []
    turns = []
    for _ in range(n_games):
        score1, score2, t = simulate_game(strat1, strat2, stop_at_7)
        turns.append(t)
        scores_1.append(score1)
        scores_2.append(score2)

    return scores_1, scores_2, turns

In [168]:
def simulate_all_strats(base, opps, n_games):
    results = {}
    for name, opp in opps.items():
        for rule in ('stop_at_7', 'all_cards'):
            stop_at_7 = (rule == 'stop_at_7')
            sc1, sc2, turns = simulate_n_games(base, opp, stop_at_7, n_games)
            results[(name, rule)] = {
                'score': sc1,
                'opp_score': sc2,
                'turns': turns
            } 
    return pd.DataFrame(results).T

In [169]:
# устанавливаем сид и выбираем стратегии
np.random.seed(150)
base = generate_strategy()
opps = {
    'reverse': reverse_strategy(base),
    'swap_paired' : swap_paired_strategy(base),
    'swap_unpaired': swap_unpaired_strategy(base)
}
# генерируем выборки
all_strats = simulate_all_strats(base, opps, 50000)

## Промежуточные результаты

In [170]:
def calculate_winrate(scores_1, scores_2):
    wins = [int(s1 > s2) for s1, s2 in zip(scores_1, scores_2) if s1 != s2]
    win_rate = 0
    if len(wins) > 0:
        win_rate = sum(wins) / len(wins)
    return win_rate

In [171]:
def simulate_all_strats(opps, all_strats):
    results = {}
    for name in opps.keys():
        for rule in ('stop_at_7', 'all_cards'):
            sc1 = all_strats.loc[(name, rule), 'score']
            sc2 = all_strats.loc[(name, rule), 'opp_score']
            turns = all_strats.loc[(name, rule), 'turns']
            win_rate = calculate_winrate(sc1, sc2)
            results[(name, rule)] = {
                'win_rate': win_rate,
                'avg_turns': sum(turns) / len(turns)
            } 
    return pd.DataFrame(results).T

In [172]:
simulate_all_strats(opps, all_strats)

win_rate  avg_turns
reverse       stop_at_7  0.211815   36.73594
              all_cards  0.044634   75.46500
swap_paired   stop_at_7  0.468177   41.55252
              all_cards  0.467659   59.47052
swap_unpaired stop_at_7  0.492868   41.01638
              all_cards  0.478406   55.73716

## Проверка гипотез

Поскольку данные **не** распределены нормально, и есть только два исхода (не считаем ничьи), я воспользовался Z-тестом.

In [173]:
def calculate_stat(scores1, scores2):
    wins = [bool(s1 > s2) for s1, s2 in zip(scores1, scores2) if s1 != s2]
    stat, pval = proportions_ztest(sum(wins), len(wins), value=0.5)
    return stat, pval

In [174]:
def calculate_stat_all_strats(opps, all_strats):
    results = {}
    for name in opps.keys():
        for rule in ('stop_at_7', 'all_cards'):
            sc1 = all_strats.loc[(name, rule), 'score']
            sc2 = all_strats.loc[(name, rule), 'opp_score']
            turns = all_strats.loc[(name, rule), 'turns']
            win_rate = calculate_winrate(sc1, sc2)
            stat, pval = calculate_stat(sc1, sc2)
            results[(name, rule)] = {
                'win_rate': win_rate,
                'avg_turns': sum(turns) / len(turns),
                'stat': stat,
                'pval': pval,
                'result': pval > 0.05
            } 
    return pd.DataFrame(results).T

In [175]:
calculate_stat_all_strats(opps, all_strats)

win_rate avg_turns        stat      pval result
reverse       stop_at_7  0.211815  36.73594 -152.835332       0.0  False
              all_cards  0.044634    75.465 -487.790436       0.0  False
swap_paired   stop_at_7  0.468177  41.55252  -13.646275       0.0  False
              all_cards  0.467659  59.47052   -14.00333       0.0  False
swap_unpaired stop_at_7  0.492868  41.01638   -3.063993  0.002184  False
              all_cards  0.478406  55.73716    -9.34431       0.0  False

## Бутстрап

In [176]:
def bootstrap_turns(turns, n_boot=10000, alpha=0.05):
    arr = np.array(turns)
    boot = []
    for _ in range(n_boot):
        samp = np.random.choice(arr, len(arr), True) 
        boot.append(np.median(samp))
    lower = np.percentile(boot, 100 * alpha / 2)
    upper = np.percentile(boot, 100 * (1 - alpha / 2))
    return lower, upper

In [177]:
def bootstrap_turns_all_strats(opps, all_strats):
    results = {}
    for name in opps.keys():
        for rule in ('stop_at_7', 'all_cards'):
            turns = all_strats.loc[(name, rule), 'turns']
            lower, upper = bootstrap_turns(turns)
            results[(name, rule)] = {
                'lower_bound': lower,
                'upper_bound': upper
            } 
    return pd.DataFrame(results).T

In [178]:
bootstrap_turns_all_strats(opps, all_strats)

lower_bound  upper_bound
reverse       stop_at_7         37.0         37.0
              all_cards         75.0         76.0
swap_paired   stop_at_7         42.0         42.0
              all_cards         59.0         59.0
swap_unpaired stop_at_7         41.0         41.0
              all_cards         56.0         56.0